In [12]:
import numpy as np
import pandas as pd
import requests
from io import StringIO

def transform_url(url: str):
    return StringIO(requests.get(url).content.decode('utf-8'))

base_url = 'https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/'
url_hackathon_scouting = f'{base_url}hackathon_scouting.csv'
url_hackathon_nwhl = f'{base_url}hackathon_nwhl.csv'
url_hackathon_womens = f'{base_url}hackathon_womens.csv'
    
scouting = pd.read_csv(transform_url(url_hackathon_scouting))
nwhl = pd.read_csv(transform_url(url_hackathon_nwhl))
womens = pd.read_csv(transform_url(url_hackathon_womens))
# https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_scouting.csv
# https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_nwhl.csv
# https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2021/main/hackathon_womens.csv

In [49]:
scouting['Event'].unique()

array(['Faceoff Win', 'Puck Recovery', 'Incomplete Play', 'Play',
       'Dump In/Out', 'Zone Entry', 'Shot', 'Takeaway', 'Goal',
       'Penalty Taken'], dtype=object)

# Data

## Events
Based on the event type, other columns will have a different meaning. X, Y, Player, Player2, and Details columns will differ for each unique value in Event column.

**Notes**: 
- Blank in Player 2 means there have no assist.
- Blank in any Detail column indicates that there are no relevant detail for that specific entry.

### Shot: Unsuccesful shot attempts
- Player: Shooter
- Player 2: Passer
- X,Y: Release Location
- Detail 1: Indicates Shot type
- Deatil 2: Shot Destiation which will on net, missed, or blocked
- Detail 3: Traffic, Binary either True or False
- Detail 4: One time, Binary either True or false

### Goal: Successfull shot attempts

- Player: Shooter
- Player 2: Passer (blank for unassisted shots)
- X,Y Coordinate: Release location of the puck
- Detail 1: Shot Type (Deflection, Fan, Slapshot, Snapshot, Wrap around, Wristshot)
- Detail 2: Shot destination (on net, missed or blocked)
- Detail 3: Traffic (true or false)
- Detail 4: One timer (true or false)

### Play: Pass attempts that are successful
- Player: Passer
- Player 2: Intended pass target
- X,Y Coordinate: Pass target location and Pass release location
- Detail 1: Either Direct (eg. a tape-to-tape pass) or Indirect (eg. a pass that is rimmed around the boards)

### Incomplete Play: Pass attempts that are unsuccessful

- Player: Passer
- Player 2: Intended pass target
- X,Y Coordinate: Pass release location and Pass target location
- Detail 1: Direct (eg. a tape-to-tape pass) or Indirect (eg. a pass that is rimmed around the boards)

### Takeaway: Any action causing change in possession

- Player: Skater credited with the takeaway
- X,Y Coordinate: Location where the skater gained possession when taking the puck away

### Puck Recovery: Possession gains initiated by retrieving a loose puck 

- Player: Skater who recovered the puck
- X,Y Coordinate: Location where skater gained possession

### Dump In/Out: Actions in which a skater intentionally concedes possession by advancing the puck up ice

- Player: Skater who dumped/advanced the puck
- X,Y Coordinate: Location where skater released the puck
- Detail 1: Possession Outcome (Retained, Lost)

### Zone Entry: Attempts to move the puck into the offensive zone from the neutral zone

- Player: Entry skater
- Player 2: Targeted defender
- X,Y Coordinate: Point of release for dumps/advances, point where puck crossed the blueline for passes and carries
- Detail 1: Entry Type (Carried, Dumped, Played)

### Faceoff Win: Faceoffs
- Player: Skater who won the draw
- Player 2: Skater who lost the draw
- X,Y Coordinate: Location of faceoff dot

### Penalty Taken: Infractions
- Player: Skater who took the penalty
- Player 2: Skater who drew the penalty
- X,Y Coordinate: Location of infraction
- Detail 1: Infraction Type (e.g. Slashing, Tripping, Roughing, Hooking, ...)

In [39]:
scouting[scouting['Home Team'] == 'Erie Otters']

,game_date,Home Team,Away Team,Period,Clock,Home Team Skaters,Away Team Skaters,Home Team Goals,Away Team Goals,Team,...,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,2019-09-20,Erie Otters,Sudbury Wolves,1,20:00,5,5,0,0,Sudbury Wolves,...,Faceoff Win,100,43,Forehand,NaN,NaN,NaN,Connor Lockhart,NaN,NaN
1,2019-09-20,Erie Otters,Sudbury Wolves,1,19:58,5,5,0,0,Sudbury Wolves,...,Puck Recovery,67,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-09-20,Erie Otters,Sudbury Wolves,1,19:53,5,5,0,0,Sudbury Wolves,...,Incomplete Play,45,49,Direct,NaN,NaN,NaN,Blake Murray,74.0,35.0
3,2019-09-20,Erie Otters,Sudbury Wolves,1,19:52,5,5,0,0,Erie Otters,...,Puck Recovery,89,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-09-20,Erie Otters,Sudbury Wolves,1,19:51,5,5,0,0,Erie Otters,...,Incomplete Play,82,49,Direct,NaN,NaN,NaN,Connor Lockhart,94.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75868,2020-03-08,Erie Otters,Saginaw Spirit,3,0:07,6,5,3,4,Erie Otters,...,Puck Recovery,177,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75869,2020-03-08,Erie Otters,Saginaw Spirit,3,0:06,6,5,3,4,Erie Otters,...,Play,184,7,Direct,NaN,NaN,NaN,Hayden Fowler,190.0,14.0
75870,2020-03-08,Erie Otters,Saginaw Spirit,3,0:04,6,5,3,4,Erie Otters,...,Play,193,23,Direct,NaN,NaN,NaN,Maxim Golod,138.0,26.0
75871,2020-03-08,Erie Otters,Saginaw Spirit,3,0:03,6,5,3,4,Erie Otters,...,Shot,137,34,Slapshot,Blocked,t,f,NaN,NaN,NaN


In [43]:
scouting['Event'].unique()

array(['Faceoff Win', 'Puck Recovery', 'Incomplete Play', 'Play',
       'Dump In/Out', 'Zone Entry', 'Shot', 'Takeaway', 'Goal',
       'Penalty Taken'], dtype=object)

In [41]:
scouting.iloc[:10,6:]

,Away Team Skaters,Home Team Goals,Away Team Goals,Team,Player,Event,X Coordinate,Y Coordinate,Detail 1,Detail 2,Detail 3,Detail 4,Player 2,X Coordinate 2,Y Coordinate 2
0,5,0,0,Sudbury Wolves,Blake Murray,Faceoff Win,100,43,Forehand,NaN,NaN,NaN,Connor Lockhart,NaN,NaN
1,5,0,0,Sudbury Wolves,Emmett Serensits,Puck Recovery,67,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,0,0,Sudbury Wolves,Emmett Serensits,Incomplete Play,45,49,Direct,NaN,NaN,NaN,Blake Murray,74.0,35.0
3,5,0,0,Erie Otters,Hayden Fowler,Puck Recovery,89,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0,0,Erie Otters,Hayden Fowler,Incomplete Play,82,49,Direct,NaN,NaN,NaN,Connor Lockhart,94.0,46.0
5,5,0,0,Sudbury Wolves,Emmett Serensits,Puck Recovery,64,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5,0,0,Sudbury Wolves,Emmett Serensits,Play,6,60,Indirect,NaN,NaN,NaN,Blake Murray,29.0,2.0
7,5,0,0,Sudbury Wolves,Blake Murray,Puck Recovery,59,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5,0,0,Sudbury Wolves,Blake Murray,Play,82,50,Indirect,NaN,NaN,NaN,Owen Gilhula,82.0,54.0
9,5,0,0,Sudbury Wolves,Owen Gilhula,Dump In/Out,102,71,Lost,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
scouting['Away Team'].unique()

array(['Sudbury Wolves', 'Erie Otters', 'Saginaw Spirit',
       'Hamilton Bulldogs', 'Guelph Storm', 'Mississauga Steelheads',
       'London Knights', 'Windsor Spitfires', 'Sarnia Sting',
       'Kitchener Rangers', 'Niagara Ice Dogs', 'Flint Firebirds',
       'Owen Sound Attack', 'Barrie Colts', 'North Bay Battalion'],
      dtype=object)

In [26]:
scouting.iloc[1000:1100, :5]

,game_date,Home Team,Away Team,Period,Clock
1000,2019-09-20,Erie Otters,Sudbury Wolves,2,9:05
1001,2019-09-20,Erie Otters,Sudbury Wolves,2,9:03
1002,2019-09-20,Erie Otters,Sudbury Wolves,2,9:01
1003,2019-09-20,Erie Otters,Sudbury Wolves,2,9:00
1004,2019-09-20,Erie Otters,Sudbury Wolves,2,9:00
...,...,...,...,...,...
1095,2019-09-20,Erie Otters,Sudbury Wolves,2,6:08
1096,2019-09-20,Erie Otters,Sudbury Wolves,2,6:05
1097,2019-09-20,Erie Otters,Sudbury Wolves,2,6:02
1098,2019-09-20,Erie Otters,Sudbury Wolves,2,5:58


In [37]:
nwhl['Home Team'].unique()

array(['Minnesota Whitecaps', 'Buffalo Beauts', 'Metropolitan Riveters',
       'Boston Pride', 'Connecticut Whale', 'Toronto Six'], dtype=object)

In [38]:
nwhl['Away Team'].unique()

array(['Boston Pride', 'Connecticut Whale', 'Toronto Six',
       'Buffalo Beauts', 'Metropolitan Riveters', 'Minnesota Whitecaps'],
      dtype=object)